In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

# SKlearn (denk om "pip install scikit-learn" en NIET "sklearn"):
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [ ]:
# load the data
url = 'titanic-train.csv'
df = pd.read_csv(url)
#df.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']

In [ ]:
# inspect the data
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
# Check de class variabele, de variabele die we willen voorspellen a.d.h.v. de features
df['Survived'].value_counts().plot(kind='bar')

In [ ]:
## Check missing values
# We kunnen makkelijk checken op missing values met de isnull methode van pandas
df.isnull().sum()
# de set bevat missing values op Age en Cabin (en Embarked)

In [ ]:
# Kan ook grafisch
sns.set_style("whitegrid")

sns.heatmap(df.isnull(), yticklabels=False, cmap="viridis")

In [ ]:
df.info()

In [ ]:
# plot the data
df.Age.hist(bins=20, label='leeftijd')
plt.legend()

In [ ]:
# data is wijd verspreid, dus hoe data missing values in te vullen?
# Is er een relatie met klasse?
sns.boxplot(x='Pclass', y='Age', data=df, palette='hls', hue='Pclass', legend=False)

Grof gezegd kunnen we zeggen dat hoe jonger een passagier is, hoe groter de kans is dat hij in de 3e klas zit. Hoe ouder een passagier is, hoe groter de kans dat hij in de 1e klas zit. Er is dus een losse relatie tussen deze variabelen. Laten we dus een functie schrijven die de leeftijd van de passagier benadert, op basis van hun klasse. Uit de boxplot blijkt dat de gemiddelde leeftijd van de 1e klas passagiers ongeveer 37 is, de 2e klas passagiers 29 is en de 3e klas passagiers 24 is.

Laten we dus een functie schrijven die elke nulwaarde in de variabele `Age` vindt en voor elke nul de waarde van de `Pclass` controleert en een leeftijdwaarde toekent op basis van de gemiddelde leeftijd van passagiers in die klasse.

In [ ]:
def age_approx(cols):
    Age = cols.iloc[0]
    Pclass = cols.iloc[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

In [ ]:
df['Age'] = df[['Age', 'Pclass']].apply(age_approx, axis=1)
df.isnull().sum()

In [ ]:
df.head(10)

In [ ]:
# waarschijnlijk doen naam, ticket en cabin er niet toe, deze kunnen we verwijderen
df = df.drop(['Name','Ticket','Cabin'], axis=1)
df.isnull().sum()

In [ ]:
# verwijderen van de rijen met de twee waardes nul in de Embarked zal niet veel uitmaken
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
#Sex en Embarked moeten numeriek gemaakt worden
df['Sex'] = df['Sex'].map({'female': '1', 'male': '0'})

df.head()

In [ ]:
df.Embarked.value_counts().plot(kind='barh')

In [ ]:
df['Embarked'] = df['Embarked'].map({'C': '1', 'S': '2', 'Q': '3'})
df.head()

In [ ]:
df.info()

In [ ]:
df['Sex'] = pd.to_numeric(df['Sex']) # alt.: df['Sex'] = df['Sex'].astype('int')
df['Embarked'] = pd.to_numeric(df['Embarked'])
df.info()

In [ ]:
df.head()

In [ ]:
# get X (features) en y (class/category)
y = np.array(df['Survived'])

X = np.array(df.iloc[:,2:9]) # slice is 'up to but not including'
X.shape

In [ ]:
# normaliseer (gemiddelde eraf en delen door de standaarddeviatie, zodat het op een normaalverdeling gaat lijken)
def normalize(X):
    scaler = StandardScaler()
    scaler = scaler.fit(X)
    X = scaler.transform(X)
    return X

X = normalize(X)

In [ ]:
X

## Na al deze verkennings- en preparatiestappen gaan we eindelijk het model maken, trainen en evalueren!

In [ ]:
# split: 70% trainingsdata en 30% testdata
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=42)

# train model
logReg = LogisticRegression()
logReg.fit(X_train, y_train)
y_pred = logReg.predict(X_test)

In [ ]:
# evalueer
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
print(classification_report(y_test, y_pred))

# Extra

Correlaties zijn een goede manier om te kijken welke features relevant zijn voor Survived en welke niet.

In [ ]:
corr_matrix = df.corr()

In [ ]:
corr_matrix

In [ ]:
corr_matrix['Survived']

## Grafisch

In [ ]:
sns.heatmap(corr_matrix, annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')

## Klopt dit met de gewichten die het LogReg-model gevonden heeft?

In [ ]:
weights = logReg.coef_
weights

In [ ]:
weights = weights[0].tolist()
weights

In [ ]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
fw = zip(weights, features)
tuple(fw)

Class en Sex zijn naar verwachting hoog. Age is hoger dan verwacht, Fare lager.